# Evaluation Metrics:

## Proposed Interface:

We will assume that we have a vector of sample **statements** and a vector of sample **responses**. We will also assume that we can generate the **proposed response** with a function `gen_pro_res`. 

In [1]:
import numpy as np

statements = [["Hello", "."], ["Goodbye", "."], ["What","is","your","name","?"], ["What", "is", "wrong", "?"]]
responses = [["Hello", "."], ["Goodbye", "."], ["Jack","."],["I", "can't", "hear", "you", "."]]

def gen_pro_res(statement):
    return ["I", "can't", "hear", "you", "."]


## Automatic Evaluation Metrics:

### BLEU:
The **bilingual evaluation understudy** an automatic evaluation metric used to measure the performance of machine generated translations and similar tasks. The high level idea is to count the number of matching **$n$-grams** between a **candidate response** and a **reference response**. The metric will output a score between $0$ and $1$ with $1$ being a perfect score.

See https://www.aclweb.org/anthology/P02-1040.pdf and https://en.wikipedia.org/wiki/BLEU for more information.

We will use NLTK (Natural Language Toolkit) which has implemented a bleu evaluation function for us. Note that BLEU ideally has a set of reference responses but can be used with a single response.

In [5]:
import nltk.translate.bleu_score as bleu_score

In [6]:
reference = [['this', 'is', 'a', 'test']]
candidate = ['this', 'is', 'a', 'test']
score = bleu_score.sentence_bleu(reference, candidate, smoothing_function=bleu_score.SmoothingFunction().method1)
print(score)

1.0


We may or may not want to use a corpus version which seems to do the same in our case. We care about dialogue responses over all sample statements. We may want to trim out punctuation. We also note that this metric doesn't directly care about the statement (but we assume the statement when comparing candidate and reference).

Calculate a single corpus-level BLEU score (aka. system-level BLEU) for all 
    the hypotheses and their respective references.  

    Instead of averaging the sentence level BLEU scores (i.e. marco-average 
    precision), the original BLEU metric (Papineni et al. 2002) accounts for 
    the micro-average precision (i.e. summing the numerators and denominators
    for each hypothesis-reference(s) pairs before the division).

In [4]:
references = [[['this', 'is', 'a', 'test']], [['josh']]]
candidates = [['this', 'is', 'a', 'test'], ['charlie']]
score = bleu_score.corpus_bleu(references, candidates)
print(score)

0.668740304976422


We may also be worried about short responses which do not contain at least one 4-gram (since BLEU looks at 1 to 4 grams). Then we would need to pick a smoothing function. See https://www.nltk.org/api/nltk.translate.html#nltk.translate.bleu_score.SmoothingFunction and http://acl2014.org/acl2014/W14-33/pdf/W14-3346.pdf.

In [5]:
score = [0]*len(statements)
for idx in range(0, len(statements)):
    print((responses[idx], gen_pro_res(statements[idx])))
    score[idx] = bleu_score.sentence_bleu([responses[idx]], gen_pro_res(statements[idx]), smoothing_function=bleu_score.SmoothingFunction().method1)
    print(score[idx])

print("Average score over all cases: ", np.average(score))
    
references = [[resp] for resp in responses]
candidates = [gen_pro_res(statement) for statement in statements]
print("Corpus score of all responses: ", bleu_score.corpus_bleu(references, candidates, smoothing_function=bleu_score.SmoothingFunction().method1))
    

(['Hello', '.'], ['I', "can't", 'hear', 'you', '.'])
0.05372849659117709
(['Goodbye', '.'], ['I', "can't", 'hear', 'you', '.'])
0.05372849659117709
(['Jack', '.'], ['I', "can't", 'hear', 'you', '.'])
0.05372849659117709
(['I', "can't", 'hear', 'you', '.'], ['I', "can't", 'hear', 'you', '.'])
1.0
Average score over all cases:  0.2902963724433828
Corpus score of all responses:  0.28117066259517454


In the above we see that the responses are not matched well for the first 3 examples because they only share the period. However, we do see when there is an exact match, they do completely agree.

### ROUGE
The **Recall-ORiented Understudy for Gisting Evaluation** is a metric often used for evaluation text summarization. There are many different variation but we will focus on ROUGE-L which compares **LCS (longest common subsequences)** in the reference and candidate responses and ROUGE-n which also looks at n-grams. A key difference between ROUGE and BLEU Is that ROUGE will look at precision, recall and F1 scores. That is, it will tell us about what appears in the reference but not candidate as well as what appears in candidate but not reference. 

See https://www.freecodecamp.org/news/what-is-rouge-and-how-it-works-for-evaluation-of-summaries-e059fb8ac840/ and https://stackoverflow.com/questions/38045290/text-summarization-evaluation-bleu-vs-rouge for more information.

We will use easy-rouge which is a completely python implementation of the ROUGE script. See https://pypi.org/project/easy-rouge/.

QUESTION: Should we implement or research a corpus level version or just "average"?

In [6]:
from rouge.rouge import rouge_n_sentence_level

for idx in range(0, len(statements)):
    candidate = responses[idx]
    reference = gen_pro_res(statements[idx])
    print(candidate, reference)
    
    # We will consider the 1-gram version
    recall, precision, rouge = rouge_n_sentence_level(candidate, reference, 1)
    print('ROUGE-1-R:', recall)
    print('ROUGE-1-P:', precision)
    print('ROUGE-1-F1:', rouge)
    
    # We will consider the 4-gram version
    recall, precision, rouge = rouge_n_sentence_level(candidate, reference, 4)
    print('ROUGE-4-R:', recall)
    print('ROUGE-4-P:', precision)
    print('ROUGE-4-F1:', rouge)

['Hello', '.'] ['I', "can't", 'hear', 'you', '.']
ROUGE-1-R: 0.2
ROUGE-1-P: 0.5
ROUGE-1-F1: 0.28571428571428575
ROUGE-4-R: 0.0
ROUGE-4-P: 0.0
ROUGE-4-F1: 0.0
['Goodbye', '.'] ['I', "can't", 'hear', 'you', '.']
ROUGE-1-R: 0.2
ROUGE-1-P: 0.5
ROUGE-1-F1: 0.28571428571428575
ROUGE-4-R: 0.0
ROUGE-4-P: 0.0
ROUGE-4-F1: 0.0
['Jack', '.'] ['I', "can't", 'hear', 'you', '.']
ROUGE-1-R: 0.2
ROUGE-1-P: 0.5
ROUGE-1-F1: 0.28571428571428575
ROUGE-4-R: 0.0
ROUGE-4-P: 0.0
ROUGE-4-F1: 0.0
['I', "can't", 'hear', 'you', '.'] ['I', "can't", 'hear', 'you', '.']
ROUGE-1-R: 1.0
ROUGE-1-P: 1.0
ROUGE-1-F1: 1.0
ROUGE-4-R: 1.0
ROUGE-4-P: 1.0
ROUGE-4-F1: 1.0


In [7]:
from rouge.rouge import rouge_l_sentence_level

for idx in range(0, len(statements)):
    candidate = responses[idx]
    reference = gen_pro_res(statements[idx])
    print(candidate, reference)
    
    # We will consider the 1-gram version
    recall, precision, rouge = rouge_l_sentence_level(candidate, reference)
    print('ROUGE-1-R:', recall)
    print('ROUGE-1-P:', precision)
    print('ROUGE-1-F1:', rouge)



['Hello', '.'] ['I', "can't", 'hear', 'you', '.']
ROUGE-1-R: 0.2
ROUGE-1-P: 0.5
ROUGE-1-F1: 0.28571428571428575
['Goodbye', '.'] ['I', "can't", 'hear', 'you', '.']
ROUGE-1-R: 0.2
ROUGE-1-P: 0.5
ROUGE-1-F1: 0.28571428571428575
['Jack', '.'] ['I', "can't", 'hear', 'you', '.']
ROUGE-1-R: 0.2
ROUGE-1-P: 0.5
ROUGE-1-F1: 0.28571428571428575
['I', "can't", 'hear', 'you', '.'] ['I', "can't", 'hear', 'you', '.']
ROUGE-1-R: 1.0
ROUGE-1-P: 1.0
ROUGE-1-F1: 1.0


### METEOR
The **Metric for Evaluation of Translation with Explicit ORdering** is also a metric for machine translation problems. It uses stemming and synonyms to match words/tokens in between the reference and candidates. Then it computes the harmonic mean based on precision and recall of the number of words matches. 

See https://en.wikipedia.org/wiki/METEOR.

In [8]:
import nltk
nltk.download('wordnet')
import nltk.translate.meteor_score as meteor_score

score = [0]*len(responses)
for idx in range(0, len(responses)):
    print((responses[idx], gen_pro_res(statements[idx])))
    resp = ' '.join(responses[idx])
    cand = ' '.join(gen_pro_res(statements[idx]))
    score[idx] = meteor_score.meteor_score(resp,cand)
    print(score[idx])

print("Average score over all cases: ", np.average(score))

[nltk_data] Downloading package wordnet to /Users/charlie/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


(['Hello', '.'], ['I', "can't", 'hear', 'you', '.'])
0.35714285714285715
(['Goodbye', '.'], ['I', "can't", 'hear', 'you', '.'])
0.35714285714285715
(['Jack', '.'], ['I', "can't", 'hear', 'you', '.'])
0.35714285714285715
(['I', "can't", 'hear', 'you', '.'], ['I', "can't", 'hear', 'you', '.'])
0.35714285714285715
Average score over all cases:  0.35714285714285715


Looks like nltk doesn't support corpus level evaluation (which we may not want) and so we could also use the java program which may support more features and be better. See https://github.com/nltk/nltk/issues/2365 and https://www.cs.cmu.edu/~alavie/METEOR/README.html#integration.



In [2]:
import os
import subprocess

os.makedirs("temp",exist_ok=True)
f_ref = open('temp/ref.dat', 'w+')
f_cand = open('temp/cand.dat', 'w+')

for idx in range(0, len(statements)):
    print(' '.join(responses[idx]))
    f_ref.write(' '.join(responses[idx]))
    print(' '.join(gen_pro_res(statements[idx])))
    f_cand.write(' '.join(gen_pro_res(statements[idx])))
    
f_ref.close()
f_cand.close()
    
meteor_cmd = ['java', '-jar', '-Xmx2G', 'meteor-1.5.jar', \
                'temp/ref.dat', 'temp/ref.dat', '-l', 'en', '-norm']
meteor_output = subprocess.run(meteor_cmd, capture_output=True)

Hello .
I can't hear you .
Goodbye .
I can't hear you .
Jack .
I can't hear you .
I can't hear you .
I can't hear you .


In [3]:
print(meteor_output.stderr.decode())
#print(meteor_output.stdout.decode())

output = {}
for line in meteor_output.stdout.decode().split('\n'):
    if ': ' in line:
        key,value = line.split(': ')
        output[key] = value

precision = float(output['Precision'])
print("Precision: ", precision)

recall = float(output['Recall'])
print("Recall: ", recall)

f1 = float(output['f1'])
print("f1: ", f1)

fmean = float(output['fMean'])
print("fmean: ", fmean)

score = float(output['Final score'])
print("Score: ", score)


Precision:  1.0
Recall:  1.0
f1:  1.0
fmean:  1.0
Score:  1.0


### WER/TER
**Word error rate** and **translation error rate** are both metrics that measure the edit distance between the hypothesis and reference outputs. See https://en.wikipedia.org/wiki/Word_error_rate and http://mt-archive.info/AMTA-2006-Snover.pdf. From what I can tell, the only real difference is that TER can handle multiple references.

In [13]:
# Code taken from: https://martin-thoma.com/word-error-rate-calculation/
# def wer(r, h):
#     """
#     Calculation of WER with Levenshtein distance.

#     Works only for iterables up to 254 elements (uint8).
#     O(nm) time ans space complexity.

#     Parameters
#     ----------
#     r : list
#     h : list

#     Returns
#     -------
#     int

#     Examples
#     --------
#     >>> wer("who is there".split(), "is there".split())
#     1
#     >>> wer("who is there".split(), "".split())
#     3
#     >>> wer("".split(), "who is there".split())
#     3
#     """
#     # initialisation
#     import numpy
#     d = numpy.zeros((len(r)+1)*(len(h)+1), dtype=numpy.uint8)
#     d = d.reshape((len(r)+1, len(h)+1))
#     for i in range(len(r)+1):
#         for j in range(len(h)+1):
#             if i == 0:
#                 d[0][j] = j
#             elif j == 0:
#                 d[i][0] = i

#     # computation
#     for i in range(1, len(r)+1):
#         for j in range(1, len(h)+1):
#             if r[i-1] == h[j-1]:
#                 d[i][j] = d[i-1][j-1]
#             else:
#                 substitution = d[i-1][j-1] + 1
#                 insertion    = d[i][j-1] + 1
#                 deletion     = d[i-1][j] + 1
#                 d[i][j] = min(substitution, insertion, deletion)

#     return d[len(r)][len(h)]

# Code taken from https://web.archive.org/web/20171215025927/http://progfruits.blogspot.com/2014/02/word-error-rate-wer-and-word.html
SUB_PENALTY = 1
INS_PENALTY = 1
DEL_PENALTY = 1

def wer(ref, hyp ,debug=False):
    r = ref.split()
    h = hyp.split()
    #costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
 
    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3
     
    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL
         
    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS
     
    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1
                 
                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL
                 
    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
        print("OP\tREF\tHYP")
        lines = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if debug:
                lines.append("OK\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            if debug:
                lines.append("SUB\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            if debug:
                lines.append("INS\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if debug:
                lines.append("DEL\t" + r[i]+"\t"+"****")
    if debug:
        lines = reversed(lines)
        for line in lines:
            print(line)
        print("#cor " + str(numCor))
        print("#sub " + str(numSub))
        print("#del " + str(numDel))
        print("#ins " + str(numIns))
    return (numSub + numDel + numIns) / (float) (len(r))
#     wer_result = round( (numSub + numDel + numIns) / (float) (len(r)), 3)
#     return {'WER':wer_result, 'Cor':numCor, 'Sub':numSub, 'Ins':numIns, 'Del':numDel}

In [14]:
# from jiwer import wer

resp_cat = [' '.join(resp) for resp in responses]
cand_cat = [' '.join(gen_pro_res(stat)) for stat in statements]
print(resp_cat)
print(cand_cat)

error = [0]*len(cand_cat)
for idx in range(0,len(statements)):
    print(resp_cat[idx])
    print(cand_cat[idx])
    error[idx] = wer(resp_cat[idx], cand_cat[idx], debug=False)
print("Error: ", error)
print("Average Error: ", np.average(error))

['Hello .', 'Goodbye .', 'Jack .', "I can't hear you ."]
["I can't hear you .", "I can't hear you .", "I can't hear you .", "I can't hear you ."]
Hello .
I can't hear you .
Goodbye .
I can't hear you .
Jack .
I can't hear you .
I can't hear you .
I can't hear you .
Error:  [2.0, 2.0, 2.0, 0.0]
Average Error:  1.5


We note that in the above, we are using the response as the ground truth and so we may end up with 'error' that is above 1.

## Human Metrics

We will want to do human metrics that measure and compare our models.

### Author's Opinion
When doing the writeup we will want to each sum our overall opinion of the model's performance. This corresponds to those metrics with larger sample groups that would ask for a rating of how well a response matches the character/emotion in question.

### General behavior

Using test data from the more general corpus, we will design questions for us to answer with the goal of deciding if a response is human appropriate.

On a scale from 1 to 10, how well does the following response match the original statement?

Statement: "I'm home!"

Response: "Welcome!"

Score: 10


Statement: "I'm home!"

Response: "It is cold."

Score: 2




Which of the following seems like the best human response?

Statement: "I'm home!"

Responses:
1. "Welcome!" (generated by our model)
2. "Welcome home." (response from conversation data)

Answer: 1


Statement: "I'm home!"

Responses:
1. "Dog!" (generated by our model)
2. "Welcome home." (response from conversation data)

Answer: 2

### General behavior

Using test data from the TV corpus, we will design questions for us to answer with the goal of deciding if a response is character appropriate.

On a scale from 1 to 10, how well does the following response match the original statement with respect to the character in question?

Statement: "I'm home!"

Response: "Hey buddy!"

Score: 10


Statement: "I'm home!"

Response: "Welcome back, old friend."

Score: 5

Which of the following seems like the best character response?

Statement: "I'm home!"

Responses:
1. "I didn't do it!" (generated by our model)
2. "Welcome home." (response from conversation data)

Answer: 1


Statement: "I'm home!"

Responses:
1. "Get out!" (generated by our model)
2. "Welcome home." (response from conversation data)

Answer: 2

### Compare
We can also compare all our models with or without the actual text in both cases to gather information on which model behaves the best.

Which of the following seems like the best character/human response?

Statement: "I'm home!"

Responses:
1. "I didn't do it!" (generated by our model a)
2. "Welcome home." (generated by our model b)
3. "you" (generated by our model c)

Answer: 2


We will want to each answer a lot of these questions (I'm thinking like 100+). Then we will want to analyze the results. It may be easier to also ask true/false version of the first questions instead of scale from 1 to 10 but we can always average the score to get some metric. 

Would be nice if we can find a few more people to also answer questions but maybe not as much.